In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 343.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00


In [59]:
!pip install ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 33.2 MB/s eta 0:00:00


In [28]:
!pip install langchain pypdf


In [57]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.llms import CTransformers

In [29]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents
data = load_pdf("/content/")

In [30]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

In [31]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [32]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.038338541984558105, 0.12346471846103668, -0.02864297851920128]

In [33]:
db = FAISS.from_documents(docs, embeddings)

In [36]:
question = "What is paracetamol?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

histamine (a compound released in allergic inflammatoryreactions) at the H
1receptor sites, responsible for imme-
diate hypersensitivity reactions such as sneezing anditching . Members of this class of drugs may also be used
for their side effects, including sedation and antiemesis
(prevention of nausea and vomiting ).
GALE ENCYCLOPEDIA OF MEDICINE 2 276AntihistaminesGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 276


In [37]:
# Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert" tokenizer.
tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")

# Create a question-answering model object by loading the pretrained "Intel/dynamic_tinybert" model.
model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

In [38]:
# Specify the model name you want to use
model_name = "Intel/dynamic_tinybert"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

In [39]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = db.as_retriever()

In [41]:
docs = retriever.get_relevant_documents("What is fever?")
print(docs[0].page_content)

(38°–40°C). In addition, a general ill feeling, muscleaches, headache , chills, and loss of appetite may be felt.
Diagnosis
If lymphangitis is suspected, the person should call
his or her doctor immediately or go to an emergencyroom. Acute lymphangitis could be diagnosed by thefamily doctor, infectious disease specialist, or an emer-gency room doctor. The painful, red streaks just belowthe skin surface and the high fever are diagnostic of acutelymphangitis. A sample of blood would be taken for cul-ture to determine whether the bacteria have entered thebloodstream. A biopsy (removal of a piece of infectedtissue) sample may be taken for culture to identify whichKEY TERMS
Biopsy —The process which removes a sample of
diseased or infected tissue for microscopic exami-nation to aid in diagnosis.
Lymphatic system —A component of the immune
system consisting of vessels and nodes. Wastematerials from organs drain into the lymphaticvessels and are filtered by the lymph nodes.


In [42]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [48]:
question = "what is Malignant?"
result = qa.run({"query": question})
print(result["result"][0])

ValueError: Context information is below. 
------------
Malignant —The term literally means growing worse
and resisting treatment. It is used as a synonym forcancerous and connotes a harmful condition thatgenerally is life-threatening.
Morphology —Literally, the study of form. In medi-
cine, morphology refers to the size, shape andstructure rather than the function of a given organ.As a diagnostic imaging technique, ultrasound facil-itates the recognition of abnormal morphologies assymptoms of underlying conditions.
GALE ENCYCLOPEDIA OF MEDICINE 2 3Abdominal ultrasoundGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 3
------------
Given the context information and not prior knowledge, answer the question: what is Malignant?
 argument needs to be of type (SquadExample, dict)